<a href="https://colab.research.google.com/github/ethanrom/LSTM-Sinhala-Folk-Poem-Generation/blob/main/sinhala_folk_poem_LSTMipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow
!pip install keras

In [ ]:
import tensorflow as tf
from keras_preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import numpy as np 

In [ ]:
# read the input file and store the poems as a list of strings
with open("input.txt", "r") as file:
    text = file.read().split("\n")

In [ ]:
# split the poems into 4 lines each and store in a new list
lines = []
for poem in text:
    poem = poem.split("\n")
    for line in poem:
        lines.append(line)

In [ ]:
# build the tokenizer on the lines
tokenizer = Tokenizer()

def tokenize(lines):
    # fit the tokenizer on the lines
    tokenizer.fit_on_texts(lines)
    return tokenizer

In [ ]:
# convert the lines to sequences
def get_sequence(tokenizer, max_length, lines):
    sequences = tokenizer.texts_to_sequences(lines)
    sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')
    return sequences

In [ ]:
# get the vocabulary size
def get_vocabulary_size(tokenizer):
    return len(tokenizer.word_index) + 1

In [ ]:
#generate lines
def generate_text(model, tokenizer, max_length, seed_text, num_gen_words, temperature=1.5):
    # Initialize the generated text
    generated_text = seed_text
    # Convert the seed text to numerical representation
    input_sequence = [tokenizer.word_index[word] for word in seed_text.split()]
    input_sequence = pad_sequences([input_sequence], maxlen=max_length, padding='pre')
    # Get the prediction for the seed text
    preds = model.predict(input_sequence, verbose=0)[0]
    # Set the temperature
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    # Sample the next word using the temperature-modified predictions
    next_index = np.argmax(np.random.multinomial(1, preds, 1))
    next_word = list(tokenizer.word_index.keys())[list(tokenizer.word_index.values()).index(next_index)]
    generated_text += " " + next_word
    # Generate the remaining words
    line_count = 1
    for i in range(num_gen_words-1):
        input_sequence = [tokenizer.word_index[word] for word in generated_text.split()[-max_length:]]
        input_sequence = pad_sequences([input_sequence], maxlen=max_length, padding='pre')
        preds = model.predict(input_sequence, verbose=0)[0]
        preds = np.asarray(preds).astype('float64')
        preds = np.log(preds) / temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)
        next_index = np.argmax(np.random.multinomial(1, preds, 1))
        next_word = list(tokenizer.word_index.keys())[list(tokenizer.word_index.values()).index(next_index)]
        generated_text += " " + next_word
        if next_word == ".":
            line_count += 1
        if line_count == 4:
            break
    return generated_text

In [ ]:
# set the hyperparameters
max_length = 8 
num_gen_words = 10 

In [ ]:
# get the tokenizer
tokenizer = tokenize(lines)

In [ ]:
# get the vocabulary size
vocabulary_size = get_vocabulary_size(tokenizer)

In [ ]:
# get the sequences
sequences = get_sequence(tokenizer, max_length, lines)

In [ ]:
# split the sequences into input and output
sequences = np.array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]

In [ ]:
# one hot encode the target
y = tf.keras.utils.to_categorical(y, num_classes=vocabulary_size)

In [ ]:
#split the data into train and test sets
total_data = X.shape[0]
split_point = int(total_data * 0.8)
X_train, X_test = X[:split_point], X[split_point:]
y_train, y_test = y[:split_point], y[split_point:]

In [ ]:
#define the model option 1
model = Sequential()
model.add(Embedding(vocabulary_size, 10, input_length=max_length - 1))
model.add(LSTM(50))
model.add(Dropout(0.1))
model.add(Dense(vocabulary_size, activation='softmax'))
print(model.summary())

In [ ]:
#define the model option 2
model = Sequential()
model.add(Embedding(vocabulary_size, 200, input_length=max_length-1))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocabulary_size, activation='softmax'))
print(model.summary())

In [ ]:
#define the model option 3
model = Sequential()
model.add(Embedding(vocabulary_size, 100, input_length=max_length-1))
model.add(LSTM(100))
model.add(Dense(vocabulary_size, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 7, 100)            54900     
                                                                 
 lstm (LSTM)                 (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 549)               55449     
                                                                 
Total params: 190,749
Trainable params: 190,749
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
#compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
#set the early stopping criteria alt
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')

In [ ]:
#fit the model alt
model.fit(X, y, batch_size=32, epochs=10, verbose=1, callbacks=[earlystop])

Epoch 1/10
5/6 [========================>.....] - ETA: 0s - loss: 2.9575 - accuracy: 0.4062

6/6 [==============================] - 0s 28ms/step - loss: 2.9535 - accuracy: 0.4085
Epoch 2/10
5/6 [========================>.....] - ETA: 0s - loss: 2.8577 - accuracy: 0.4437

6/6 [==============================] - 0s 38ms/step - loss: 2.8727 - accuracy: 0.4451
Epoch 3/10
6/6 [==============================] - ETA: 0s - loss: 2.7957 - accuracy: 0.4573

6/6 [==============================] - 0s 41ms/step - loss: 2.7957 - accuracy: 0.4573
Epoch 4/10
4/6 [===================>..........] - ETA: 0s - loss: 2.6674 - accuracy: 0.4688

6/6 [==============================] - 0s 18ms/step - loss: 2.7186 - accuracy: 0.4573
Epoch 5/10
4/6 [===================>..........] - ETA: 0s - loss: 2.6022 - accuracy: 0.4609

...

6/6 [==============================] - 0s 15ms/step - loss: 2.5804 - accuracy: 0.4695
Epoch 7/10
5/6 [========================>.....] - ETA: 0s - loss: 2.4982 - accuracy: 0.5125

6/6 [==============================] - 0s 16ms/step - loss: 2.5105 - accuracy: 0.5061
Epoch 8/10
5/6 [========================>.....] - ETA: 0s - loss: 2.4468 - accuracy: 0.5688

6/6 [==============================] - 0s 15ms/step - loss: 2.4412 - accuracy: 0.5671
Epoch 9/10
5/6 [========================>.....] - ETA: 0s - loss: 2.3591 - accuracy: 0.5875

6/6 [==============================] - 0s 16ms/step - loss: 2.3726 - accuracy: 0.5854
Epoch 10/10
5/6 [========================>.....] - ETA: 0s - loss: 2.2958 - accuracy: 0.6250

6/6 [==============================] - 0s 15ms/step - loss: 2.3042 - accuracy: 0.6159


In [ ]:
#evaluate the model alt
_, accuracy = model.evaluate(X, y, verbose=0)
print("Accuracy: %.2f" % (accuracy*100))

Accuracy: 62.20


In [ ]:
#generate a poem

seed_text = "සඳුන් සුවඳ විහිදෙයි වට"
generated_poem = generate_text(model, tokenizer, max_length-1, seed_text, num_gen_words)
print("Generated Poem:")
print(seed_text + generated_poem)

Generated Poem:
සඳුන් සුවඳ විහිදෙයි වටසඳුන් සුවඳ විහිදෙයි වට තැනින් කමතට බාන්නේ සොඳිනේ කමතා කරන්නේ වෙහෙසෙනවා පොකුණු ළඟ පපුව


In [ ]:
#generate a poem
seed_text = "සඳුන් සුවඳ විහිදෙයි වට"
generated_poem = generate_text(model, tokenizer, max_length-1, seed_text, num_gen_words)

# Split the generated text into words
words = generated_poem.split()

print("Generated Poem:")
# Iterate over the first 4 words and print each line
for i in range(4):
    print(" ".join(words[i:i+4]))

Generated Poem:
සඳුන් සුවඳ විහිදෙයි වට
සුවඳ විහිදෙයි වට අදින්නේ
විහිදෙයි වට අදින්නේ කමතට
වට අදින්නේ කමතට කරන්නේ
